[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your_repo/role_lora_training.ipynb)

# Role-LoRA Fine-Tuning Notebook
This notebook fine-tunes GPT-Neo-125M with LoRA.  
*If a file named `wandb_key.txt` (containing only your Weights & Biases API key) is present, logging will be enabled.  Otherwise W&B is disabled automatically.*

In [ ]:
# --- Install dependencies ---
!pip install -q transformers datasets peft accelerate bitsandbytes wandb

In [ ]:
# --- W&B auto-setup ---
import os
wandb_key_file = 'wandb_key.txt'
if os.path.isfile(wandb_key_file):
    with open(wandb_key_file) as f:
        key = f.read().strip()
    if key:
        import wandb
        wandb.login(key=key)
        print('Weights & Biases logging ENABLED')
    else:
        os.environ['WANDB_DISABLED'] = 'true'
        print('wandb_key.txt is empty – W&B disabled.')
else:
    os.environ['WANDB_DISABLED'] = 'true'
    print('No wandb_key.txt – W&B disabled.')

In [ ]:
BASE_MODEL = "EleutherAI/gpt-neo-125M"   # ~500 MB

In [ ]:
# --- Build tiny toy dataset ---
from datasets import Dataset

examples = [
    {"role": "Detective", "text": "Role: Detective\nUser: Describe a crime scene.\nAssistant: The alley smelled of rust and secrets, every shadow a possible clue."},
    {"role": "Poet", "text": "Role: Poet\nUser: Write about the sunrise.\nAssistant: Dawn spills saffron light across the quiet roofs of the city."},
    {"role": "Comedian", "text": "Role: Comedian\nUser: Tell a joke about cats.\nAssistant: Why did the cat join Instagram? To get more pawsitive feedback!"}
]
dataset = Dataset.from_list(examples)

In [ ]:
# --- Tokenize ---
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, max_length=512)

tokenized = dataset.map(tokenize_fn)

In [ ]:
# --- Load base model + LoRA ---
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    device_map="auto"
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn"],  # correct module for GPT-Neo
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
# --- Training ---
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="role-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized
)

trainer.train()

In [ ]:
# --- Save LoRA adapter ---
model.save_pretrained("role-lora/adapter")
tokenizer.save_pretrained("role-lora/adapter")

In [ ]:
# --- Inference helper ---
from peft import PeftModel

base = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")
lora = PeftModel.from_pretrained(base, "role-lora/adapter")

def generate(role, user_prompt):
    full_prompt = f"Role: {role}\nUser: {user_prompt}\nAssistant:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(lora.device)
    out = lora.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.8,
        top_p=0.9
    )
    print(tokenizer.decode(out[0], skip_special_tokens=True))

generate("Poet", "Write about an autumn forest.")
generate("Detective", "Describe a mysterious stranger.")